In [ ]:
# 1302223032 - Muhammad Fajar Mufid
# 1302220093 - Rd. M. Faisal Ramadhan Junaidi
# Tugas / Casebased Reasoning

# Menggunakan fuzzy-based System Mandani, maka defuzzificationnya menggunakan metode Center of Gravity (COG)

import pandas as pd

# Fungsi keanggotaan trapesium
# Fungsi Trapesium dipilih karena cocok untuk merepresentasikan konsep-konsep "sekitar" atau "rentang tertentu".
def trapezoid(x, a, b, c, d):
    if x <= a:
        return 0
    elif a < x < b:
        return (x - a) / (b - a)
    elif b <= x <= c:
        return 1
    elif c < x < d:
        return (d - x) / (d - c)
    else:
        return 0

# Fuzzifikasi Pelayanan (nilai 1-100)
# untuk 3 himpunan, yakni himpunan buruk, cukup, dan baik
def fuzzify_service(x):
    return {
        'Buruk': trapezoid(x, 0, 0, 30, 40),
        'Cukup': trapezoid(x, 30, 40, 60, 80),
        'Baik': trapezoid(x, 60, 80, 100, 100),
    }

#Fuzzifikasi Harga (harga 20000 - 55000)
def fuzzify_price(x):
    return {
        'Murah': trapezoid(x, 20000, 25000, 30000, 40000),
        'Sedang': trapezoid(x, 30000, 40000, 45000, 50000),
        'Mahal': trapezoid(x, 45000, 50000, 55000, 55000),
    }

# Rule base fuzzy (dari Pelayann dan Harga)
rules = [
    ('Buruk', 'Mahal', 'Rendah'),
    ('Buruk', 'Sedang', 'Rendah'),
    ('Buruk', 'Murah', 'Sedang'),
    ('Cukup', 'Mahal', 'Sedang'),
    ('Cukup', 'Sedang', 'Sedang'),
    ('Cukup', 'Murah', 'Tinggi'),
    ('Baik', 'Mahal', 'Sedang'),
    ('Baik', 'Sedang', 'Tinggi'),
    ('Baik', 'Murah', 'Tinggi'),
]

# Nilai crisp kelayakan untuk defuzzifikasi (skala 1-10)
kelayakan_values = {
    'Rendah': 3,
    'Sedang': 6,
    'Tinggi': 9
}

# Inferensi fuzzy
def infer(fuzz_service, fuzz_price):
    results = []
    for serv_cat, price_cat, kel_cat in rules:
        degree = min(fuzz_service[serv_cat], fuzz_price[price_cat])
        if degree > 0:
            results.append((degree, kelayakan_values[kel_cat]))
    return results

# Defuzzifikasi dengan metode Center of Gravity (COG)
def defuzzify(rule_results):
    if not rule_results:
        return 0
    numerator = sum(deg * val for deg, val in rule_results)
    denominator = sum(deg for deg, val in rule_results)
    return numerator / denominator if denominator != 0 else 0

# Membaca data dari file restoran.xlsx
data = pd.read_excel('restoran.xlsx')

# Proses fuzzy untuk semua data restoran
results = []
for _, row in data.iterrows():
    fuzz_service = fuzzify_service(row['Pelayanan'])
    fuzz_price = fuzzify_price(row['harga'])
    rule_results = infer(fuzz_service, fuzz_price)
    score = defuzzify(rule_results)
    results.append({
        'Id': row['id Pelanggan'],
        'Pelayanan': row['Pelayanan'],
        'Harga': row['harga'],
        'Score': score
    })

# Sortir berdasarkan skor kelayakan dari terbesar
results_sorted = sorted(results, key=lambda x: x['Score'], reverse=True)

# Ambil 10 restoran terbaik
top5 = results_sorted[:10]

# Simpan hasil ke file Excel peringkat.xlsx
df_out = pd.DataFrame(top5)
df_out.to_excel('peringkat.xlsx', index=True)

# Print hasil 10 teratas
print(df_out)